In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# NLTK 데이터 다운로드
nltk.download('stopwords')

# 데이터 불러오기
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 데이터 확인
print(train_data.head())

# 데이터 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # 토큰화 및 불용어 제거
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    # 스테밍
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# 텍스트 데이터 전처리
train_data['text'] = train_data['text'].apply(preprocess_text)
test_data['text'] = test_data['text'].apply(preprocess_text)

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(train_data['text']).toarray()
y = train_data['target']

# 테스트 데이터 처리
X_test = tfidf.transform(test_data['text']).toarray()

# 데이터 분할 (훈련/검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의 및 학습
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# 검증 데이터로 예측
y_pred = model.predict(X_val)

# F1 스코어 평가
f1 = f1_score(y_val, y_pred)
print(f'Validation F1 Score: {f1:.4f}')
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측 및 제출 파일 생성
test_data['target'] = model.predict(X_test)
submission = test_data[['id', 'target']]
submission.to_csv('submission_v1_lr.csv', index=False)
print("Submission file saved as 'submission.csv'")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  
Validation F1 Score: 0.7406
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       874
           1       0.81      0.68      0.74       649

    accuracy                           0.80      1523
   macro avg       0.80      0.78      0.79      1523
weighted avg       0.80      0.80      0.79      1523

Submission file saved as 'submission.csv'
